In [2]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
import os
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import IndexToString, StringIndexer
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings('ignore')


master = "local"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'
spark = SparkSession.builder.master(master).appName('Linear_Regression').getOrCreate()
lista=['eth']

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2a20a940-71a9-4d3a-9ade-a09299ec5821;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-log

In [3]:
def download(CryptoName):
    today = date.today()
    d1 = today.strftime("%Y-%m-%d")
    end_date = d1
    d2 = date.today() - timedelta(days=182)
    d2 = d2.strftime("%Y-%m-%d")
    start_date = d2
    crypto=CryptoName
    crypto=crypto+'-eur'
    data = yf.download(crypto,start=start_date, end=end_date, progress=False)
    return data

In [4]:
import shutil
import pyspark.sql.functions as funcs

from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline


for i in range(len(lista)):
    data=download(lista[i])
    data['High']= round(data['High'],2)
    data['Low']= round(data['Low'],2)
    data['Close']= round(data['Close'],2)
    df = spark.createDataFrame(data)
    df=df.drop("Open","Adj Close")
    df = df.select("Volume","High","Low","Close")

    vectorAssembler = VectorAssembler(inputCols=['Volume','High','Low'], outputCol='Attributes')
    v_df = vectorAssembler.transform(df)
    v_df = v_df.select(['Attributes','Close'])
    splits = v_df.randomSplit([0.85,0.15])
    train_df = splits[0]
    test_df = splits[1]
    lr = LinearRegression(featuresCol = 'Attributes',labelCol = 'Close', maxIter=50, regParam=0.05, elasticNetParam=0.75)
    model = lr.fit(train_df)
    #pred = model.evaluate(test_df)    
    #print(lista[i])
    #pred.predictions.show()
   

22/06/28 13:08:01 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/06/28 13:08:01 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [5]:
spark.sparkContext.addFile("./tmp.csv")
dataset_path = SparkFiles.get("tmp.csv")
current = spark.read.format('csv').option('header', True).load(dataset_path)

current.show()
current = current.select(
        funcs.col("Volume").cast('float'),
        funcs.col("High").cast('float'),
        funcs.col("Low").cast('float'),
)

current = vectorAssembler.transform(current)
current = current.select(['Attributes'])
current.show()
pred = model.transform(current)
#pred.show()
pred = pred.toPandas()
pred=pred.iloc[0]['prediction']
pred=round(pred,2)
pred

+-----------+-------+--------+
|     Volume|   High|     Low|
+-----------+-------+--------+
|17546869193|20319.0|19446.38|
+-----------+-------+--------+

+--------------------+
|          Attributes|
+--------------------+
|[1.7546868736E10,...|
+--------------------+



19775.86

In [6]:
crypto='btc'
crypto
pred

19775.86

In [7]:
final = pd.DataFrame(columns = ['symbol','prediction'])
final

,symbol,prediction


In [10]:
final = final.append({'symbol':crypto, 'prediction':pred},ignore_index=True)
final

,symbol,prediction
0,btc,19775.86
